In [26]:
from datasets import load_dataset, Dataset
import gc
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import LoraConfig, AutoPeftModelForCausalLM
from trl import SFTConfig, SFTTrainer

In [2]:
dataset = load_dataset('iamjoon/klue-mrc-ko-rag-dataset', split='train')
print(set(dataset['type']))
dataset = dataset.class_encode_column('type')

{'mrc_question_with_1_to_4_negative', 'paraphrased_question', 'mrc_question', 'no_answer', 'synthetic_question'}


In [3]:
# cast_column 메서드 활용도 가능
type_name_dict = {1:'mrc_question_with_1_to_4_negative', 3:'paraphrased_question', 2:'no_answer', 4:'synthetic_question', 0:'mrc_question'}

In [4]:
system_message = '''
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
----------
{search_result}'''

In [5]:
print('원본 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {dataset['type'].count(type_name)}')

원본 데이터의 type 분포:
mrc_question: 491
mrc_question_with_1_to_4_negative: 296
no_answer: 404
paraphrased_question: 196
synthetic_question: 497


In [6]:
split_dataset = dataset.train_test_split(test_size=0.5, stratify_by_column='type')
train_dataset_format = split_dataset['train']
test_dataset_format = split_dataset['test']

In [7]:
def format_data(sample):
    search_result = '\n------\n'.join(f'문서{idx+1}: {result}' for idx, result in enumerate(sample['search_result']))

    return {
        'messages':[{'role':'system', 'content':system_message.format(search_result=search_result)},
                    {'role':'user', 'content':sample['question']},
                    {'role':'assistant', 'content':sample['answer']}]
    }

In [8]:
train_dataset = [format_data(train_data) for train_data in train_dataset_format]
test_dataset = [format_data(test_data) for test_data in test_dataset_format]

In [9]:
print(f'\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개')
print('--'*20)
print('\n학습 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {train_dataset_format['type'].count(type_name)}')
print('\n테스트 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {test_dataset_format['type'].count(type_name)}')


전체 데이터 분할 결과: Train 942개, Test 942개
----------------------------------------

학습 데이터의 type 분포:
mrc_question: 246
mrc_question_with_1_to_4_negative: 148
no_answer: 202
paraphrased_question: 98
synthetic_question: 248

테스트 데이터의 type 분포:
mrc_question: 245
mrc_question_with_1_to_4_negative: 148
no_answer: 202
paraphrased_question: 98
synthetic_question: 249


In [10]:
train_dataset[345]['messages']

[{'role': 'system',
  'content': '\n당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n----------\n문서1: 지난 9월 서울 성동구 금호동에서 분양된 ‘e편한세상 신금호’와 ‘힐스테이트 금호’ 아파트. 이들 단지 모델하우스는 성동구가 아닌 강남구 청담동과 도곡동에 각각 설치됐다. 강남 지역으로 이동하기 쉬운 입지 여건이라는 점에 착안해 강남권 주택 수요자를 공략하기 위해서였다. 두 단지 모두 청약 1순위에서 평균 20 대 1 안팎의 경쟁률로 조기에 ‘완판(완전 판매)’됐다.‘표적 판촉(타깃 마케팅)’에 나서는 건설회사들이 많아지고 있다. 아파트 분양 소재지에 일률적으로 견본주택을 두던 예전 방식에서 벗어나 잠재 수요자가 있는 곳을 찾아내 모델하우스를 설치하는 게 대표적이다. 뿐만아니라 실수요자 의견을 아파트 평면 등 상품 설계에도 적극 반영하고 있다.대림산업은 내달 초 위례신도시(서울 송파구, 경기 성남·하남에 걸쳐 조성)에 분양할 예정인 기업형 임대주택(뉴 스테이) ‘e편한세상 테라스 위례’(360가구) 모델하우스를 위례신도시가 아닌 강남구 신사동에 설치할 예정이다. 강남권 대체 주거지로 조성되는 위례신도시 특징에 맞춰 실제 구매 의

In [11]:
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)

In [13]:
model_id = 'Qwen/Qwen2.5-3B-Instruct'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [14]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=8,
    bias='none',
    target_modules=['q_proj', 'v_proj'],
    task_type='CAUSAL_LM'
)

In [15]:
args = SFTConfig(
    output_dir='qwen_2.5-3b_rag-ko',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim='adamw_torch_fused',
    logging_steps=10,
    save_strategy='steps',
    save_steps=100,
    bf16=True,
    learning_rate=1e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type='constant',
    push_to_hub=False,
    remove_unused_columns=False,
    report_to=None,
    completion_only_loss=True
)

In [16]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    peft_config=peft_config
)

Tokenizing train dataset:   0%|          | 0/942 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/942 [00:00<?, ? examples/s]

In [17]:
trainer.train()
trainer.save_model()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
c:\Coding\Local\exercise_book_ver2\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.210100
20,2.032600
30,1.821200
40,1.698000
50,1.605000
60,1.654600
70,1.577600
80,1.632900
90,1.563600
100,1.563000


c:\Coding\Local\exercise_book_ver2\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Coding\Local\exercise_book_ver2\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return 

In [ ]:
del model
del tokenizer
del trainer
gc.collect()
torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [18]:
prompt_list = []
label_list = []

for prompt in test_dataset['messages']:
    text = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=False)
    
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]

    prompt_list.append(input)
    label_list.append(label)


In [22]:
print(prompt_list[36])
print('--'*30)
print('--'*30)
print('--'*30)

<|im_start|>system

당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
----------
문서1: “공무원이 와서 직접 환경영향평가를 해달라. 왜 우리가 일일이 찾아다니도록 하느냐.”3일 청와대에서 열린 제2차 규제개혁장관회의에서 소개된 귀농인 이희숙 씨의 사연은 한국의 행정 규제가 창업과 투자를 어떻게 옥죄고 있는지를 적나라하게 보여줬다. 대통령, 총리, 부총리, 환경부 장관, 법제처장 등이 뒤섞여 난상토론을 벌였지만 예비 기업인의 답답함을 해소해주기에는 역부족이었다.이씨의 하소연은 지역 농산물인 찹쌀과 오미자로 한과를 생산하기 위해 가공시설을 지으려고 하는데 현행 상수원보호 관련 조항 때문에 허가가 나지 않는다는 것이었다. 농한기에만 공장을 돌릴 예정이기 때문에 폐수 배출량도 일반 가정과 다를 바 없는데 지나친 규제라는 것이다. 법령에 대한 “국토교통부와 환경부의 해석도 달라 혼란스럽다”고 했다.윤성규 환경부 장관은 이에 “비슷한 민원이 많은 만큼 법 개정을 거쳐 내년 중에 허가가 나도록 하겠다”고 대답했다. 그러자 박근혜 대통령이 “내년에요? 내년이면 되겠느냐”고 되물었다. 너무 늦는 것 아니냐는 반문이었다. 윤 장관은 수도법을 개정해야 하는 사안이라고 거듭 말했다.규

In [23]:
print(label_list[36])


규제 개혁을 위해 필요한 요소는 다음과 같습니다:

1. **효율적인 시스템 구축**: 규제 개혁을 위해서는 새로운 규제개혁 시스템을 만들어야 합니다. 이는 무분별하게 만들어진 규제를 없애거나 합리적으로 조정하는 데 필요합니다. 기존의 관행을 답습하지 않고 새로운 시스템을 통해 규제개선을 실질적으로 이뤄야 합니다[[ref2]].

2. **관행의 변화**: 과거의 관행을 답습하지 않고 새로운 접근 방식을 도입해야 합니다. 규제 개혁이 관료 주도로 이루어질 경우 실적이나 숫자 맞추기에 급급할 수 있으며, 이는 국민이 변화를 실감하지 못하게 할 뿐 아니라 정부의 신뢰를 떨어뜨릴 수 있습니다[[ref2]].

3. **사람의 변화**: 규제 개혁은 대통령 한 사람의 의지로만 해결되는 것이 아니며, 관련된 모든 사람들이 협력하여 변화를 이끌어내야 합니다. 이는 규제 개혁의 실질적인 효과를 높이는 데 중요합니다[[ref2]].

4. **국제적 규범 준수**: 국제적 규범에 어긋나는 규제는 과감히 없애야 합니다. 대기업과 수도권에 대한 무분별한 규제는 철폐하는 것이 바람직하며, 이는 한국의 규제 환경을 국제 기준에 맞추는 데 도움이 됩니다[[ref2]].

5. **융통성 발휘**: 규제 해석에 있어 부처 간의 차이를 줄이고 융통성을 발휘해야 합니다. 이는 규제의 일관성을 높이고, 기업들이 규제 때문에 겪는 혼란을 줄이는 데 도움이 됩니다[[ref1]].

6. **과도한 규제 철폐**: 기업가 정신을 위축시키는 과도한 규제와 법 집행을 개선해야 합니다. 이는 기업 활동을 촉진하고 경제 활력을 높이는 데 중요합니다[[ref3]].

7. **입법부의 신중한 접근**: 입법부가 새로운 법을 내놓기 전에 모든 사실을 충분히 고려하고 결정을 내려야 합니다. 이는 불필요한 규제를 줄이고, 규제 개혁의 효과를 높이는 데 도움이 됩니다[[ref5]].

이러한 요소들을 종합적으로 고려하여 규제 개혁을 추진하면, 기업 활동을 촉진하고 경제 성장을 도모할 수 있을 것입니다.<|im

In [29]:
base_model_id = 'Qwen/Qwen2.5-3B-Instruct'
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map='auto', dtype=torch.float16)
base_tokenizer = AutoTokenizer.from_pretrained(base_model_id)
pipe = pipeline('text-generation', model=base_model, tokenizer=base_tokenizer)
eos_token = base_tokenizer('<|im_end|>', add_special_tokens=False)['input_ids'][0]

def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    
    return outputs[0]['generated_text'][len(prompt):].strip()

prompt = prompt_list[36]
label = label_list[36]
pred = test_inference(pipe, prompt)

print(f'모델의 예측:\n{pred}')
print(f'정답:\n{label}')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


모델의 예측:
규제 개혁을 위해 필요한 요소는 여러 가지가 있습니다. 먼저, 대통령이나 정부의 의지가 중요합니다. 규제 개혁은 단순히 대통령 한 사람의 의지로 이루어지는 것이 아니라, 의회 입법을 통해 무분별하게 만들어지는 규제를 없애거나 조정하는 과정이 필요합니다. 또한, 규제 개혁 시스템을 새로 만들어야 하며, 이를 위해서는 시스템, 관행, 사람 등 세 가지 실타래를 풀어야 합니다.

문서2에서 제시된 내용을 보면, 규제 개혁은 의회 입법을 통해 무분별하게 만들어지는 규제를 없애거나 조정하는 과정이 필요하다는 점이 명확히 제시되어 있습니다. 또한, 규제 개혁 시스템을 새로 만들어야 한다는 의견도 제시되어 있습니다. 이러한 내용은 규제 개혁을 위한 중요한 요소들입니다.

문서3에서도 규제 개혁 시스템을 새로 만들어야 한다는 의견이 제시되어 있습니다. 이는 규제 개혁을 위한 중요한 요소 중 하나입니다. 또한, 규제 개혁 시스템을 새로 만들기 위해서는 시스템, 관행, 사람 등 세 가지 실타래를 풀어야 한다는 의견도 제시되어 있습니다.

따라서, 규제 개혁을 위해 필요한 요소는 다음과 같습니다: 
- 대통령이나 정부의 의지
- 의회 입법을 통해 무분별하게 만들어지는 규제를 없애거나 조정하는 과정
- 규제 개혁 시스템을 새로 만들어야 한다는 의견
- 시스템, 관행, 사람 등 세 가지 실타래를 풀어야 한다는 의견

위의 요소들을 모두 갖추면 규제 개혁을 효과적으로 수행할 수 있을 것입니다. [[ref1], [ref2], [ref3]]
정답:

규제 개혁을 위해 필요한 요소는 다음과 같습니다:

1. **효율적인 시스템 구축**: 규제 개혁을 위해서는 새로운 규제개혁 시스템을 만들어야 합니다. 이는 무분별하게 만들어진 규제를 없애거나 합리적으로 조정하는 데 필요합니다. 기존의 관행을 답습하지 않고 새로운 시스템을 통해 규제개선을 실질적으로 이뤄야 합니다[[ref2]].

2. **관행의 변화**: 과거의 관행을 답습하지 않고 새로운 접근 방식을 도입해야 합니다. 규제 

In [32]:
del base_model
del base_tokenizer
gc.collect()
torch.cuda.empty_cache()

In [36]:
peft_model_id = 'qwen_2.5-3b_rag-ko/checkpoint-236'
peft_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map='auto', dtype=torch.float16)
peft_tokenizer = AutoTokenizer.from_pretrained('qwen_2.5-3b_rag-ko')
pipe = pipeline('text-generation', model=peft_model, tokenizer=peft_tokenizer)

prompt = prompt_list[36]
label = label_list[36]
pred = test_inference(pipe, prompt)

print(f'모델의 예측:\n{pred}')
print(f'정답:\n{label}')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cpu


모델의 예측:
규제 개혁을 위해 필요한 요소는 다음과 같습니다:

1. 정부의 의지와 책임: 정부가 규제 개혁을 위해 적극적으로 나서야 합니다. 대통령, 총리, 부총리, 장관 등이 규제 개혁을 추진하고 지시해야 합니다. 또한, 법제처장, 법무부 장관 등이 규제 개혁을 주도할 수 있도록 권한을 부여해야 합니다.

2. 규제 개혁 시스템: 규제 개혁을 위한 시스템을 마련해야 합니다. 이 시스템은 규제 개혁을 위한 의결 절차, 규제 개혁의 목표 설정, 규제 개혁의 실행 계획 수립, 규제 개혁의 평가 및 보고 등의 과정을 포함해야 합니다.

3. 규제 개혁의 전문가 참여: 규제 개혁을 위한 전문가의 참여가 중요합니다. 전문가들은 규제 개혁의 효과를 높이는 데 도움이 됩니다. 전문가들은 규제 개혁의 효과를 높이기 위해 규제 개혁의 목표 설정, 규제 개혁의 실행 계획 수립, 규제 개혁의 평가 및 보고 등의 과정에서 전문적인 지식과 경험을 활용할 수 있습니다.

4. 규제 개혁의 효과를 높이는 법안: 규제 개혁을 위한 법안이 필요합니다. 이 법안은 규제 개혁의 목표를 명확히 설정하고, 규제 개혁의 실행 계획을 명확히 설정하고, 규제 개혁의 평가 및 보고를 명확히 설정합니다.

5. 규제 개혁의 효과를 높이는 법제처장: 법제처장은 규제 개혁을 위한 법안의 통과를 돕습니다. 법제처장은 법안의 통과를 위해 필요한 모든 작업을 수행합니다. 이는 법안의 입법 과정에서 법제처장이 중요한 역할을 합니다.

6. 규제 개혁의 효과를 높이는 법원: 법원은 규제 개혁을 위한 법안의 통과를 돕습니다. 법원은 법안의 통과를 위해 필요한 모든 작업을 수행합니다. 이는 법원이 규제 개혁을 위한 법안의 통과를 돕는 역할을 합니다.

7. 규제 개혁의 효과를 높이는 기업인: 기업인들은 규제 개혁을 위한 법안의 통과를 돕습니다. 기업인들은 규제 개혁을 위한 법안의 통과를 위해 필요한 모든 작업을 수행합니다. 이는 기업인들이 규제 개혁을 위한 법안의 통과를 돕는 역할을 합니다.

8. 규제 개혁의 효과를

In [37]:
del peft_model
del peft_tokenizer
gc.collect()
torch.cuda.empty_cache()